<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/Recontato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Autenticação google
from google.colab import auth
auth.authenticate_user()
print('Autenticado')

# Importar biblioteca biqquery
from google.cloud import bigquery

import datetime as dt
import pandas as pd


Autenticado


In [ ]:
# projeto
maga_bigdata = 'maga-bigdata'
#autenticação e select
client = bigquery.Client(project=maga_bigdata)
sql = """with cte as ( -- sem duplicidade de id
        select    id_chamada, data_hora_chamada ,grupo_de_trabalho,nome_do_agente , numero_do_pedido , status_do_pedido,
                  `maga-bigdata.sac.fn_validador_cpf_cnpj`(safe_cast(`maga-bigdata.sac.fn_replace_caracteres`(cpf_ou_cnpj_do_cliente) as int64)) as cpf_cnpj_string              
              from `maga-bigdata.sac.call_detail_hist_union`
              where grupo_de_trabalho in ('QuerodeCasamento','Market Place','Hibrido','Entrega Prazo 3P','Entrega Prazo 1P','Posvendas','Liberacao','Recontato','Recontato 1P','Recontato 3P','Recontato Liberacao')
                  and if(chamada_saida_entrada = 'Manual',0,1) = 1
                   and  `maga-bigdata.sac.fn_validador_cpf_cnpj`(safe_cast(`maga-bigdata.sac.fn_replace_caracteres`(cpf_ou_cnpj_do_cliente) as int64)) is not null
                           
),

-- base com row_number primeira chamada atendida por um agente
primeira_chamada_agente as ( select   id_chamada as id_chamada_1, data_hora_chamada as data_hora_chamada_1 ,
                                      grupo_de_trabalho as grupo_de_trabalho_1 ,nome_do_agente as nome_do_agente_1,
                                      cpf_cnpj_string as cpf_cnpj_string_1,numero_do_pedido as numero_do_pedido_1, status_do_pedido as status_do_pedido_1,
                                    row_number() over(partition by cast(data_hora_chamada as date),cpf_cnpj_string  order by data_hora_chamada asc) as contato_1
                             from cte
                             where nome_do_agente is not null and nome_do_agente not in ('-')
                                   ),

-- base populada com  recontato após o atendimento com o agente
base_1 as   (                          
              select a.*,
             
              b.id_chamada as id_chamada_2, b.data_hora_chamada as data_hora_chamada_2,
              b.grupo_de_trabalho as grupo_de_trabalho_2,b.nome_do_agente as nome_do_agente_2, b.cpf_cnpj_string as cpf_cnpj_string_2,
              b.numero_do_pedido as numero_do_pedido_2, b.status_do_pedido as status_do_pedido_2,
              row_number() over(partition by a.id_chamada_1 order by b.data_hora_chamada asc) as primeiro_contato_depois_2        
             
              from primeira_chamada_agente as a
              left join cte as b on a.cpf_cnpj_string_1 = b.cpf_cnpj_string
                                                  and cast(a.data_hora_chamada_1 as date) = cast(b.data_hora_chamada as date)
                                                  and b.data_hora_chamada > a.data_hora_chamada_1
              where a.contato_1  =  1
            ),


-- base seleciona apenas o primeiro recontato após a chamada do agente
base_analitica as (

                    select a.*,
                                count(b.id_chamada) as qtd_apos_2,
                                sum(case when b.nome_do_agente is null and b.id_chamada is not null  then 1 when b.nome_do_agente = '-' and  b.id_chamada is not null then 1 else 0 end) as sum_nao_agent_2,
                                sum(case when b.nome_do_agente is not null and b.nome_do_agente <> '-' then 1 else 0 end) as sum_agente_2
                   
                    from base_1 as a
                    left join cte as b on a.cpf_cnpj_string_2 = b.cpf_cnpj_string
                                                                      and cast(a.data_hora_chamada_2 as date) = cast(b.data_hora_chamada as date)
                                                                      and b.data_hora_chamada > a.data_hora_chamada_2
                   
                    where primeiro_contato_depois_2 = 1
                   
                    group by id_chamada_1, data_hora_chamada_1, grupo_de_trabalho_1 ,numero_do_pedido_1, status_do_pedido_1,
                    nome_do_agente_1 ,cpf_cnpj_string_1 ,contato_1 ,id_chamada_2 ,
                    data_hora_chamada_2, grupo_de_trabalho_2, nome_do_agente_2 ,cpf_cnpj_string_2,
                    numero_do_pedido_2, status_do_pedido_2,
                    primeiro_contato_depois_2
)



-- select base analitica
select * from base_analitica as a
where cast(a.data_hora_chamada_1 as date) between '2021-05-16' and '2021-05-17'"""

# Select salvo na variavel
base_recontato = client.query(sql).to_dataframe()

In [ ]:
base_recontato_v1 = base_recontato[['data_hora_chamada_1', 'id_chamada_1', 'id_chamada_2', 'qtd_apos_2', 'grupo_de_trabalho_2']].fillna('')
base_recontato_v1['Data']  = pd.to_datetime(base_recontato_v1['data_hora_chamada_1'], dayfirst=True)
base_recontato_v1['Data'] = base_recontato_v1['Data'].dt.strftime('%Y-%m-%d')


def condicao_mais_que_3(base_recontato_v1):                                  
    if base_recontato_v1['qtd_apos_2'] == 0:
        return 0 
    else:
        return 1

def condicao_id_chamada_2(base_recontato_v1):                                  
    if base_recontato_v1['id_chamada_2'] == '':
        return 0 
    else:
        return 1


def condicao_recontato(base_recontato_v1):
    if ("Recontato" in base_recontato_v1):
        return 'Recontato' 
    elif ("Recontato 1P" in base_recontato_v1):
        return 'Recontato'
    elif ("Recontato 3P" in base_recontato_v1):
        return 'Recontato'
    elif ("Recontato Liberacao" in base_recontato_v1):
        return 'Recontato'
    else:
      return 'Não Recontato'

                    
  

base_recontato_v1['Mais_de_3'] = base_recontato_v1.apply(condicao_mais_que_3, axis=1)
base_recontato_v1['id_chamada_2_v2'] = base_recontato_v1.apply(condicao_id_chamada_2, axis=1)
base_recontato_v1['Status_Recontato'] = base_recontato_v1['grupo_de_trabalho_2'].apply(lambda x: condicao_recontato(x))

dinamica_v1 = pd.pivot_table(base_recontato_v1, values=['id_chamada_1', 'id_chamada_2_v2', 'qtd_apos_2', 'Mais_de_3'], index=['Data'], aggfunc={'id_chamada_1':'count',
                                                                                                                                                'id_chamada_2_v2':'sum',
                                                                                                                                                'qtd_apos_2':'sum',
                                                                                                                                                'Mais_de_3':'sum'})
dinamica_v1.head()

,Mais_de_3,id_chamada_1,id_chamada_2_v2,qtd_apos_2
Data,,,,
2021-05-17,473,19328,2419,664


In [ ]:
dinamica_v2 = pd.pivot_table(base_recontato_v1, values=['qtd_apos_2'], index=['Data'],columns=['Status_Recontato'], aggfunc={'qtd_apos_2':'sum'})
dinamica_v2.head()

qtd_apos_2          
Status_Recontato Não Recontato Recontato
Data                                    
2021-05-17                 362       302

In [ ]:
base_recontato = base_recontato_v1.query('Status_Recontato=="Recontato"')
dinamica_v3 = pd.pivot_table(base_recontato, values=['Mais_de_3'], index=['Data'],columns=['grupo_de_trabalho_2'], aggfunc={'Mais_de_3':'sum'})
dinamica_v3.head()

Mais_de_3                                              
grupo_de_trabalho_2 Recontato Recontato 1P Recontato 3P Recontato Liberacao
Data                                                                       
2021-05-17                 13          114           43                  43

In [ ]:
base_recontato = base_recontato_v1.query('Status_Recontato=="Recontato"')
dinamica_v3 = pd.pivot_table(base_recontato, values=['Status_Recontato'], index=['Data'],columns='grupo_de_trabalho_2', aggfunc={'Status_Recontato':'count'})
dinamica_v3.head()

Status_Recontato  ...                    
grupo_de_trabalho_2        Recontato  ... Recontato Liberacao
Data                                  ...                    
2021-05-17                        29  ...                 244

[1 rows x 4 columns]

In [ ]:
base_recontato = base_recontato_v1.query('Status_Recontato=="Recontato"')
dinamica_v4 = pd.pivot_table(base_recontato, values=['qtd_apos_2'], index=['Data'],columns='grupo_de_trabalho_2', aggfunc={'qtd_apos_2':'sum'})
dinamica_v4.head()

qtd_apos_2                                              
grupo_de_trabalho_2  Recontato Recontato 1P Recontato 3P Recontato Liberacao
Data                                                                        
2021-05-17                  19          167           52                  64